**Website used in this project: https://www.thetvdb.com/series/the-ellen-degeneres-show**, 

**I do not own the data**(but code is definitely mine). This is for educational purpose and a project to learn web scraping using selenium. I chose this, because I love the **Ellen show**.

In [13]:
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from IPython.display import Image
import datetime

**Adding Chrome Driver path**

In [2]:
PATH ="C:\Program Files\chromedriver.exe"

**Cell below will open chrome browser**

In [3]:
driver = webdriver.Chrome(PATH)

**Adding Link of the website we want to scrape from**

In [4]:
driver.get('https://www.thetvdb.com/series/the-ellen-degeneres-show')

<i>**Note**: if you are here just for the code, scroll down to the **Final scrapper code**.

# Code to scrape the following data
- Season_episode_no
- Date_aired
- Guests_on_the_show 

<img src='./images/website 1.PNG'>

Scroll down

<img src='./images/website2.PNG'>

See the little arrow that is link for season 1 data and then there is list of 18 seasons in total

So first of all I want my scraper code to automatically go into each link of seasons list and then scrape data for each season and append the data in one single table. So by this we will have one dataset of all season with its episodes. Sounds fun right!!!

<img src='./images/inspect.PNG'> 

<img src='./images/season1.PNG'> 

**Right click** -> **inspect** -> **on RHS top bar click on arrow on right of Elements tab** -> **then select season 1** -> **you will see a portion highlighted in blue** -> **right click on it** -> **COPY** -> **copy XPATH** -> **you will get below mentioned path** copy it in one cell and do not touch it for now.


**Season 1 XPATH** : //*[@id="page-series"]/div[3]/div[2]/div[2]/div[6]/div[1]/ul/li[3]/h4/a

<img src='./images/inspect2.PNG'> 

<img src='./images/season18.PNG'> 

**Right click** -> **inspect** -> **on RHS top bar click on arrow on right of Elements tab** -> **then select last season  i.e season 18** -> **you will see a portion highlighted in blue** -> **right click on it** -> **COPY** -> **copy XPATH** -> **you will get below mentioned path** copy it in the same cell and again do not touch it.


**Season 18 XPATH** : //*[@id="page-series"]/div[3]/div[2]/div[2]/div[6]/div[1]/ul/li[20]/h4/a

So we want to iterate from Season 1 Xpath to Season 18 Xpath

**Now what is Xpath?** : XPath is a query language for selecting nodes from an XML document. So Selenium uses this and many other ways to navigate through HTML or XML document, XPATH is on of them. The other ways to select a node are using id, classes etc. but I find XPATH **headachefree** if that's a word.

Now let's get back to the task, can you spot the difference between Season1_XPATH and Season18_XPATH ?

if no, here's the answer- **//*[@id="page-series"]/div[3]/div[2]/div[2]/div[6]/div[1]/ul/ 

(till ul/)** everything is same and then after li, **h4/a** is also common in both of them.

Main game here is in **li[3]** to **li[20]**, difference= 20-3+1 =18( yes +1, as ends of the range are included **BASIC MATHS**) which is equal to total no. of seasons.

So our loop will go from 3 to 21(end of range is excluded in python) and we will get each season.


Okay, so we can access each of these season links great!!, but now what?

Now our code should be able to visit each season's link and click on it to open the page containing all the data of that season

<img src='./images/season_click_code.PNG'> 

This clicks on the link( and path to those links are given using the XPATH we saw before) and the code above can open each link on by one.

**Can it?**

No it can't , because after visiting a link , you should get back to the main page again only then it will be able to select next link of the Season list.

<img src='./images/season_loop.PNG'> 

# Do Something useful

Now just like before click on row of episode 1 and collect XPATH for **S01E01, Name, First Aired**, just like we did for Seasons and copy them in a separate cell for reference.

<img src='./images/epone.PNG'> 

ep 1:

- **season index** : //*[@id="page-season"]/div[4]/div[2]/div[1]/table/tbody/tr[1]/td[1]
- **guests**(Name): //*[@id="page-season"]/div[4]/div[2]/div[1]/table/tbody/tr[1]/td[2]/a
- **date**(First aired): //*[@id="page-season"]/div[4]/div[2]/div[1]/table/tbody/tr[1]/td[3]/div

<img src='./images/eponesixsix.PNG'> 

ep 166:
    
- **season index**: //*[@id="page-season"]/div[4]/div[2]/div[1]/table/tbody/tr[166]/td[1]
- **guest**(Name): //*[@id="page-season"]/div[4]/div[2]/div[1]/table/tbody/tr[166]/td[2]/a
- **date**(First aired): //*[@id="page-season"]/div[4]/div[2]/div[1]/table/tbody/tr[166]/td[3]/div

Now let's spot the difference between first episode's -(Season_index,guest,date XPATH) and last episode's -(Season_index,guest,date XPATH)

- season_index XPATH changes from tr[1] to tr[166] rest everything is same
- guest(Name) XPATH changes from tr[1] to tr[166] rest everything is same
- date(First aired) XPATH changes from tr[1] to tr[166] rest everything is same

Not all seasons have 166 episodes, so we can take 1 to 200 for precaution for each episode. Now you might be thinking that code will give error if we go beyond limit of no of episodes in any season. **Yes!!*, it will give **NoSuchElementException** but we will handle it using **try-except** such that if exception occurs continue from next iteration. You will understand more when you see the code.

Okay, so we can access each data column of each episode, now what?

Now our code should be able to read text associated with each XPATH and we will store them. 

<img src='./images/episode_loop.PNG'> 

# Final scrapper code: 
**Do keep the chrome browser launched by this notebook open, in front of your eyes, you will see the magic of selenium and python**

In [5]:
Date = []
Season_ep = []
Guests = []


for i in range(3,21):
    
    element=WebDriverWait(driver,20).until(EC.visibility_of_element_located((By.XPATH ,'//*[@id="page-series"]/div[3]/div[2]/div[2]/div[6]/div[1]/ul/li['+str(i)+']/h4/a')))
    element.click()
    
    for j in range(1,200):
        try:
            
            Season_ep.append(driver.find_element_by_xpath('//*[@id="page-season"]/div[4]/div[2]/div[1]/table/tbody/tr['+str(j)+']/td[1]').text)
            Guests.append(driver.find_element_by_xpath('//*[@id="page-season"]/div[4]/div[2]/div[1]/table/tbody/tr['+str(j)+']/td[2]/a').text)
            Date.append(driver.find_element_by_xpath('//*[@id="page-season"]/div[4]/div[2]/div[1]/table/tbody/tr['+str(j)+']/td[3]/div').text)
        
        except NoSuchElementException:
            
            continue  
    print('Season:'+str(i-2)+' done') 
    
    element=WebDriverWait(driver,20).until(EC.visibility_of_element_located((By.XPATH ,'//*[@id="page-season"]/div[3]/div/a[3]')))
    element.click()

Season:1 done
Season:2 done
Season:3 done
Season:4 done
Season:5 done
Season:6 done
Season:7 done
Season:8 done
Season:9 done
Season:10 done
Season:11 done
Season:12 done
Season:13 done
Season:14 done
Season:15 done
Season:16 done
Season:17 done
Season:18 done


In [6]:
len(Date),len(Guests),len(Season_ep) #checking if each column has same size of data

(3091, 3091, 3091)

**Clubbing everything together and making a dataframe**

In [7]:
d = zip(Season_ep,Date,Guests)
mapped = list(d)
df = pd.DataFrame(mapped, columns =['Season', 'Date','Guests'])

In [10]:
df

,Season,Date,Guests
0,S01E01,"September 8, 2003",Series Premiere
1,S01E02,"September 9, 2003",Justin Timberlake
2,S01E03,"September 10, 2003",Betty White/Sherri Shepherd
3,S01E04,"September 11, 2003",Megan Mullally
4,S01E05,"September 12, 2003",Dennis Quaid/Lisa Marie Presley
...,...,...,...
3086,S18E174,"June 28, 2021",Guest Host Tiffany Haddish with Tyrese Gibson ...
3087,S18E175,"June 29, 2021","Guest Host Chelsea Handler with Kristen Bell, ..."
3088,S18E176,"June 30, 2021","Guest Host Wanda Sykes with Don Cheadle, Ilana..."
3089,S18E177,"July 1, 2021",Guest Host Chelsea Handler with Robin Wright a...


**Let's verify the dataset, if it is accurate or not** 

So I remember **Deepika Padukone** went to Ellen show. If this is complete database as I believe. It should contain that entry. Lets also see who were **other guests that day**.

In [11]:
df[df['Guests'].str.contains('Deepika Padukone')]

,Season,Date,Guests
2270,S14E84,"January 18, 2017","Matt LeBlanc, Deepika Padukone"


Season 14 Episode 84, in January 18 2017, It was around release of her Hollywood debut movie **XXX:Return of Xander Cage** with Vin Diesel.

In [14]:
#Converting date to a much favourable format

df['Date'] = df['Date'].apply(lambda x: datetime.datetime.strptime(x, '%B %d, %Y').date())
df['Date'] = pd.to_datetime(df['Date'])

In [16]:
df.tail(10)

,Season,Date,Guests
3081,S18E169,2021-06-21,"Cecily Strong, Ann Dowd"
3082,S18E170,2021-06-22,"Chris “Ludacris” Bridges, James TW"
3083,S18E171,2021-06-23,"Henry Golding, Damien Caillaud, Urbain Ngobobo..."
3084,S18E172,2021-06-24,"Minnie Driver, Blake Shelton"
3085,S18E173,2021-06-25,Guest Host Stephen “tWitch” Boss with Casey Wi...
3086,S18E174,2021-06-28,Guest Host Tiffany Haddish with Tyrese Gibson ...
3087,S18E175,2021-06-29,"Guest Host Chelsea Handler with Kristen Bell, ..."
3088,S18E176,2021-06-30,"Guest Host Wanda Sykes with Don Cheadle, Ilana..."
3089,S18E177,2021-07-01,Guest Host Chelsea Handler with Robin Wright a...
3090,S18E178,2021-07-02,Guest Host Stephen “tWitch” Boss with Jay Phar...


In [ ]:
df.to_csv('Ellen_dataset(2003-2021).csv')

# At the end I just want to say, I love The Ellen Show❤.  And Each time you run this notebook, it will give you an up to date dataset of Ellen Show. But sadly this is the last season(or maybe not). But hey good thing is, we do not have to increase the season index to 22 in our for loop.